### Step 1: Run code samples in sequence to populate Firestore database

In [1]:
import pandas as pd
from google.cloud import firestore
db = firestore.Client()
import uuid

In [2]:
batch = db.batch()

df = pd.read_csv('/home/jupyter/spanner_shopify/apps.csv', sep=',', header=0, lineterminator='\n')
apps_rows = df.values.tolist()

counter = 1

for apps_row in apps_rows:

    app_record = {}
    app_record['id'] = apps_row[0]
    app_record['title'] = apps_row[1]
    app_record['developer'] = apps_row[2]
    app_record['rating'] = apps_row[6]
    app_record['review_count'] = apps_row[7]


    app_ref = db.collection('apps').document(apps_row[0])

    batch.set(app_ref, app_record)
    counter +=1 
    if counter == 399:
        batch.commit()
        counter = 1

In [21]:
batch = db.batch()

df = pd.read_csv('/home/jupyter/spanner_shopify/apps.csv', sep=',', header=0, lineterminator='\n')
apps_rows = df.values.tolist()

df = pd.read_csv('/home/jupyter/spanner_shopify/apps_categories.csv', sep=',', header=0, lineterminator='\n')
apps_category_rows = df.values.tolist()
counter =1
for apps_row in apps_rows:
    
    app_record = {}
    app_record['id'] = apps_row[0]
    app_record['title'] = apps_row[1]
    app_record['developer'] = apps_row[2]
    app_record['rating'] = apps_row[6]
    app_record['review_count'] = apps_row[7]

    
    app_ref = db.collection('apps').document(apps_row[0])
    
    batch.set(app_ref, app_record)
    
    for apps_category_row in apps_category_rows:
        if apps_row[0] == apps_category_row[0]:
            
            category_record = {}
            category_record['id'] = apps_category_row[0]
            category_record['title'] = apps_category_row[1]
            
            category_ref = app_ref.collection('category').document(apps_category_row[0])
        
            batch.set(category_ref, category_record)
    
    batch.commit()
    

In [4]:
batch = db.batch()

df = pd.read_csv('/home/jupyter/spanner_shopify/key_benefits.csv', sep=',', header=0, lineterminator='\n')
key_benefits_rows = df.values.tolist()

df = pd.read_csv('/home/jupyter/spanner_shopify/apps.csv', sep=',', header=0, lineterminator='\n')
apps_rows = df.values.tolist()
counter =1
for key_benefits_row in key_benefits_rows:
    
    key_benefits_record = {}
    key_benefits_record['id'] = key_benefits_row[0]
    key_benefits_record['title'] = key_benefits_row[1]
    key_benefits_record['review_count'] = key_benefits_row[2]

    
    key_benefits_ref = db.collection('key_benefits').document(key_benefits_row[0])
    
    batch.set(key_benefits_ref, key_benefits_record)
    unique_id = str(uuid.uuid4())
    for apps_row in apps_rows:
        if key_benefits_row[0] == apps_row[0]:
            apps_record = {}
            apps_record['id'] = apps_row[0]
            apps_record['title'] = apps_row[1]
            apps_record['developer'] = apps_row[2]
            apps_record['rating'] = apps_row[6]
            apps_record['review_count'] = apps_row[7]
           
            
            apps_ref = key_benefits_ref.collection('apps').document(apps_row[0])
        
            batch.set(apps_ref, apps_record)
    
    batch.commit()

In [27]:
batch = db.batch()

df = pd.read_csv('/home/jupyter/spanner_shopify/pricing_plan_features.csv', sep=',', header=0, lineterminator='\n')
pricing_plan_features_rows = df.values.tolist()

df = pd.read_csv('/home/jupyter/spanner_shopify/apps.csv', sep=',', header=0, lineterminator='\n')
apps_rows = df.values.tolist()
counter =1
for pricing_plan_features_row in  pricing_plan_features_rows:
    
    pricing_plan_features_record = {}
    pricing_plan_features_record['app_id'] = pricing_plan_features_row[0]
    pricing_plan_features_record['pricing_plan_id'] = pricing_plan_features_row[1]
    pricing_plan_features_record['feature'] = pricing_plan_features_row[2]

    
    pricing_plan_features_ref = db.collection('pricing_plan_features').document(pricing_plan_features_row[0])
    
    batch.set( pricing_plan_features_ref, pricing_plan_features_record)
    
    for apps_row in apps_rows:
        if pricing_plan_features_row[0] == apps_row[0]:
            
            apps_record = {}
            apps_record['id'] = apps_row[0]
            apps_record['title'] = apps_row[1]
            apps_record['developer'] = apps_row[2]
            apps_record['rating'] = apps_row[6]
            apps_record['review_count'] = apps_row[7]
            apps_ref = pricing_plan_features_ref.collection('apps').document(apps_row[0])
        
            batch.set(apps_ref, apps_record)
    
    batch.commit()

# Get a document count for each collection and subcollection you created.

In [14]:
documents = list(db.collection('apps').get())
print("# of documents in collection: {}".format(len(documents)))

# of documents in collection: 3547


In [24]:
documents = list(app_ref.collection('category').get())
print("# of documents in subcollection: {}".format(len(documents)))

# of documents in subcollection: 2


In [30]:
documents = list(db.collection('key_benefits').get())
print("# of documents in collection: {}".format(len(documents)))

# of documents in collection: 3184


In [25]:
documents = list(key_benefits_ref.collection('apps').get())
print("# of documents in subcollection: {}".format(len(documents)))

# of documents in subcollection: 1


In [29]:
documents = list(db.collection('pricing_plan_features').get())
print("# of documents in collection: {}".format(len(documents)))

# of documents in collection: 4382


In [28]:
documents = list(pricing_plan_features_ref.collection('apps').get())
print("# of documents in subcollection: {}".format(len(documents)))

# of documents in subcollection: 0


### List the 10 apps with the highest number of reviews (based on apps.review_count).
### Return the id, title, developer, rating and reviews_count of those apps.
### Order the results by reviews_count in descending order.

In [13]:
app_ref = db.collection(u'apps')
query = app_ref.order_by(u'review_count', direction=firestore.Query.DESCENDING).limit(10)
results = query.stream()
for doc in results:
    print(f'{doc.id} => {doc.to_dict()}')
    

d9f142ee-b141-4dc4-9353-173db61d2eb0 => {'developer': 'Privy ‑ Exit Pop Ups & Email', 'title': 'https://apps.shopify.com/privy', 'review_count': 23078, 'id': 'd9f142ee-b141-4dc4-9353-173db61d2eb0', 'rating': 4.7}
78ea0810-c008-4a4e-a82f-de0c790e3286 => {'title': 'https://apps.shopify.com/free-shipping-bar', 'review_count': 8737, 'id': '78ea0810-c008-4a4e-a82f-de0c790e3286', 'developer': 'Free Shipping Bar', 'rating': 4.9}
b88488b0-9912-44d3-b736-224c36f09d95 => {'rating': 4.8, 'id': 'b88488b0-9912-44d3-b736-224c36f09d95', 'title': 'https://apps.shopify.com/shoppop', 'developer': 'Sales Pop ‑ Popup Notification', 'review_count': 6905}
e528a60e-94f8-4e92-80e2-5bc6013b8283 => {'review_count': 5986, 'id': 'e528a60e-94f8-4e92-80e2-5bc6013b8283', 'developer': 'BEST Currency Converter', 'title': 'https://apps.shopify.com/doubly-currency-converter', 'rating': 4.8}
be2640c4-01b5-4d52-9f68-cae8c0734d0d => {'review_count': 5596, 'developer': 'Recart FB Messenger Marketing', 'title': 'https://apps